In [18]:
# Standard libraries
import os
import numpy as np
import pandas as pd

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
from plotnine import *
from IPython.display import display

# Machine Learning
import sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
from scipy.cluster import hierarchy as hc
from fastai.imports import *

In [19]:
data = pd.read_csv('../etc/data/lownoise/auto2.csv')
#data = pd.read_csv('../etc/data/highnoise/auto2.csv')
data.shape

(93, 23)

In [20]:
data.iloc[5,:]#Rear_seat_room, Fuel_tank_capacity, Engine_size, Class- 

maker                               Buick
 type                             Midsize
 CityMPG+                              22
 HighwayMPG+                           31
 Air_Bags_standard                      1
 Drive_train_type                       1
 Number_of_cylinders                    4
 Engine_size                          2.2
 Horsepower                           110
 RPM                                 5200
 Engine_revolutions_per_mile         2565
 manual_transmission_available          0
 Fuel_tank_capacity                  16.4
 Passenger_capacity                     6
 Length                               189
 Wheelbase                            105
 Width                                 69
 U-turn_space                          41
 Rear_seat_room                        28
 Luggage_capacity                      16
 Weight-                             2880
 domestic                               1
 Class-                              15.7
Name: 5, dtype: object

In [21]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
imp = IterativeImputer(max_iter=10, random_state=0)

In [22]:
data.replace('?', np.nan, inplace=True)
datanp = data.to_numpy()
datanp[5]

array(['Buick', 'Midsize', 22, 31, '1', '1', '4', '2.2', '110', '5200',
       '2565', 0, '16.4', '6', '189', '105', '69', '41', '28', '16', 2880,
       1, 15.7], dtype=object)

In [23]:
from sklearn.preprocessing import LabelEncoder
encoder1 = LabelEncoder()
encod_data1 = encoder1.fit_transform(datanp[:,0])
datanp[:,0] = encod_data1
datanp[5]

array([3, 'Midsize', 22, 31, '1', '1', '4', '2.2', '110', '5200', '2565',
       0, '16.4', '6', '189', '105', '69', '41', '28', '16', 2880, 1,
       15.7], dtype=object)

In [24]:
def addnan(encoder , datanp, col):
    classes1 = list(encoder.classes_)
    if np.nan in classes1:
        id = classes1.index(np.nan)
        for i,value in enumerate(datanp[:,col]):
            if value == id:
                datanp[i][col] = np.nan


In [25]:
addnan(encoder1 , datanp, 0)

In [26]:
datanp[5]

array([3, 'Midsize', 22, 31, '1', '1', '4', '2.2', '110', '5200', '2565',
       0, '16.4', '6', '189', '105', '69', '41', '28', '16', 2880, 1,
       15.7], dtype=object)

In [27]:
encoder2 = LabelEncoder()
encod_data2 = encoder2.fit_transform(datanp[:,1])
datanp[:,1] = encod_data2
datanp[22]

array([7, 3, 29, 33, '0', '1', '4', '1.5', '92', '6000', '3285', 1,
       '13.2', '5', '174', '98', '66', '32', '26.5', '11', 2270, 1, 9.2],
      dtype=object)

In [28]:
addnan(encoder2, datanp, 1)
datanp[22]

array([7, 3, 29, 33, '0', '1', '4', '1.5', '92', '6000', '3285', 1,
       '13.2', '5', '174', '98', '66', '32', '26.5', '11', 2270, 1, 9.2],
      dtype=object)

In [29]:
dataimp = imp.fit_transform(datanp)
dataimp[5]

array([3.000e+00, 2.000e+00, 2.200e+01, 3.100e+01, 1.000e+00, 1.000e+00,
       4.000e+00, 2.200e+00, 1.100e+02, 5.200e+03, 2.565e+03, 0.000e+00,
       1.640e+01, 6.000e+00, 1.890e+02, 1.050e+02, 6.900e+01, 4.100e+01,
       2.800e+01, 1.600e+01, 2.880e+03, 1.000e+00, 1.570e+01])

In [30]:
dataimp[:,0:7] = np.round(dataimp[:,0:7])
dataimp[:,8:12] = np.round(dataimp[:,8:12])
dataimp[:,13:-5] = np.round(dataimp[:,13:-5])
dataimp[:,-4:-1] = np.round(dataimp[:,-4:-1])

In [31]:
decode = encoder1.inverse_transform(dataimp[:,0].astype(np.int32))
decode1 = encoder2.inverse_transform(dataimp[:,1].astype(np.int32))
Data = np.column_stack((decode, decode1, dataimp[:,2:]))
tmp = Data[:,-2].astype(int)
newData = np.column_stack((Data[:,:-2], tmp, Data[:,-1]))
newData[5]

array(['Buick', 'Midsize', 22.0, 31.0, 1.0, 1.0, 4.0, 2.2, 110.0, 5200.0,
       2565.0, 0.0, 16.4, 6.0, 189.0, 105.0, 69.0, 41.0, 28.0, 16.0,
       2880.0, 1, 15.7], dtype=object)

In [32]:
dfnew = pd.DataFrame(newData, columns=data.columns)
dfnew.to_csv("iter_low_auto2.csv", index=False)
#dfnew.to_csv("iter_high_auto2.csv", index=False)